In [0]:
import io
import sqlite3
import pandas as pd

In [0]:
#Vocês devem baixar e descompactar cada arquivo zip desse link do meu github: https://github.com/Pendulun/TP2-IBD/tree/master/Partes
conn = sqlite3.connect('./Ocorrencias.db') # Esse é o BD. Ele vai ser gerado automaticamente no mesmo diretório que salvarem esse .ipynb
cursor = conn.cursor()

In [0]:
# Para cada arquivo descompactado, vocês vão executar isso aqui substituindo o caminho do arquivo
#Porém, porcausa das chaves estrangeiras, talvez seja necessário executar isso em uma ordem específica, sendo ela:
# ocorrencia, tipoacidente,ocorrenciaacidente,municipio,tipounidadeoperacional,unidadeoperacional,pessoa,tipoenvolvido,
# ocorrenciapessoa,tipoveiculo ,uf,ocorrenciaveiculo, veiculo,marcaveiculo,localbr (LEMBRANDO QUE UF E LOCALBR ESTÃO NO MESMO ARQUIVO)
f = io.open('./caminho do arquivo', 'r',encoding = "ISO-8859-1") 
sql = f.read()
cursor.executescript(sql)

In [0]:
#Esse comando PRAGMA está exemplificado na especificação do tp pelo professor, acredito que seja assim que executa
cursor.execute("PRAGMA cache_size = 0")

Como combinado, cada um deve pensar em pelo menos 4 consultas sendo que cada consulta deve ser feita de duas formas diferentes para testes de eficiência. Além disso, tentem fazer consultas que suprimam as necessidades das especificações:
- 2 consultas envolvendo as operações de seleção e projeção;
- 3 consultas envolvendo a junção de duas relações;
- 3 consultas envolvendo a junção de três ou mais relações;
- 2 consultas envolvendo funções de agregação sobre o resultado da junção de pelo menos duas relações.

Obviamente, se cada um precisa fazer apenas 4 consultas, não será possível cada um fazer as 10 consultas necessárias, porém, se tentarem bolar 1 consulta por requisito já fica ótimo.

In [0]:
#Esse comando é para ver o tempo que leva para executar a query. Ela também é especificada pelo professor.
#Não sei se ela deve ficar aqui, em cima da célula de consulta, ou na própria célula de consulta
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 1, versão 1",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 1, versão 2",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 2, versão 1",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 2, versão 2",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 3, versão 1",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 3, versão 2",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 4, versão 1",conn)
df

In [0]:
%%timeit

In [0]:
df = pd.read_sql_query("Consulta 4, versão 2",conn)
df